__No te olvides de rellenar esto:__

- Número de grupo: 4
- Nombre de los integrantes del grupo: Carlos Tardón Rubio y Carlos Morán Alfonso

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de lcases mediantes las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q695269' -> {'Q25630013', 'Q3388256', 'Q524526', 'Q846109', 'Q960389'} )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q34379' -> {'Q1879241', 'Q54820129'} )

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename) as f:
    data = json.load(f)
```

In [30]:
import json
import sys

def not_exist(dic,key):
    try:
        dic[key]
    except KeyError:
        dic[key] = set()


## La clase taxonomia
class taxonomia:
    def __init__(self):
        filename = "query.json"
        url = "http://www.wikidata.org/entity/"
        with open(filename,encoding='utf-8') as f: # especifico encoding para evitar errores de compilación
            data = json.load(f)
        self.etiquetas = dict()
        self.subclases = dict()
        self.superclases = dict()

        for line in data:
            key1 = line['concept'][len(url):]
            key2 = line['subconcept'][len(url):]
            self.etiquetas[key1] = line['conceptLabel']
            self.etiquetas[key2] = line['subconceptLabel']
            
            # Comprobamos todas la combinaciones posibles para evitar KeyError a la hora de añadir con dict.add
            not_exist(self.subclases,key1)
            not_exist(self.subclases,key2)
            not_exist(self.superclases,key1)
            not_exist(self.superclases,key2)
            
            self.subclases[key1].add(key2)
            self.superclases[key2].add(key1)

            

    ## Print-tree: Ejercicio 3
    def print_tree(self,concept,depth=2,initial=0):
        if initial <= depth: # Debemos imprimir el subárbol hasta una cierta profundidad
            etiq = self.etiquetas[concept]
            subclass = self.subclases[concept]
            print("\t"*initial+str(initial)+" "+etiq+" ("+concept+")")
            for sub in subclass:
                self.print_tree(sub,depth,initial+1)



    ## LCS: Ejercicio 4
    def sub(self,idClass): #Halla los conceptos más generales de un instrumento dado
        res = set()
        for sup in self.superclases[idClass]:
            res = res.union(self.sub(sup)) #Metemos los conceptos mas generales que sus superclases
        res = res.union(self.superclases[idClass]).union({idClass}) #Metemos tambien el instrumento actual y sus superclases
        return res
    
    def LCS(self,id1, id2):
        sub1 = self.sub(id1)
        sub2 = self.sub(id2)
        com_sub = sub1.intersection(sub2) # Los conceptos mas generales a ambos
        sub_sub = set()  # Vamos a calcular los mas generales de los generales a ambos
        for s in com_sub:
            generales = self.sub(s)
            generales.remove(s) # Solo queremos los estrictamente mas generales
            sub_sub = sub_sub.union(generales)
        return com_sub.difference(sub_sub) # devolvemos los instrumentos mas generales a ambos, restandoles aquellos 
                                           # que tengan un concepto mas especifico que sea mas general a ambos
    


    ## Caminos mínimos: Ejercicio 5
    def dfs(self,src,dst): # Búsqueda en profundidad guardando los vértices que vistamos
        etiq = self.etiquetas[src]
        subclass = self.subclases[src]
        vertex = etiq+" ("+src+")" 
        
        if src == dst:
            return [vertex]
        # caminos guarda todos los caminos posibles. Acabaremos devolviendo el de menor longitud resolviendo empates aleatoriamente
        caminos = set()
        for sub in subclass:
            path = self.dfs(sub,dst)
            if len(path) > 0:
                caminos.add(tuple([vertex] + path))
        if len(caminos) > 0:
            return list(min(caminos,key=len))
        return []
    

    ## Similitud: Ejercicio 6
    def similarity(self,A,B):
        raiz = 'Q34379'
        C = self.LCS(A,B)
        simil = list()
        # Guardamos todas las similitudes resultantes de todos los LCS posibles y devolvemos el que mayor similitud nos proporcione
        for elem in C:
            l1 = self.dfs(raiz,elem)
            l2 = self.dfs(elem,A)
            l3 = self.dfs(elem,B)
            sim = (len(l1) - 1) / (len(l1) + len(l2) + len(l3) - 3)
            simil.append((sim,l1,l2,l3))
        return max(simil)


### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [31]:
tax = taxonomia()

In [32]:
## Hecha en el ejercicio 2 dentro de la clase. Ejemplos de ejecución
depth = 3
tax.print_tree('Q17172850')

0 voice (Q17172850)
	1 bass (Q27911)
		2 octavist (Q7082656)
		2 bass-baritone (Q810480)
		2 basso profondo (Q2532487)
		2 character bass (Q20638448)
		2 dramatic high bass (Q54636036)
		2 heavy acting bass (Q54636271)
		2 high bass (Q54636007)
		2 Q5885030 (Q5885030)
		2 Bass bourdon (Q64363543)
		2 buffo bass (Q1002146)
		2 lyric high bass (Q3636053)
		2 serious bass (Q54636068)
	1 Treble voice (Q25303818)
	1 throat singing (Q1067089)
	1 male singing voice (Q54285279)
	1 mezzo-soprano (Q186506)
		2 light mezzo-soprano (Q54634726)
		2 mezzo-soprano castrato (Q54634945)
		2 lyric mezzo-soprano (Q1878954)
		2 Q1300059 (Q1300059)
		2 dramatic mezzo-soprano (Q6012297)
		2 character mezzo-soprano (Q54634862)
		2 sopranist (Q1999862)
		2 coloratura mezzo-soprano (Q54634572)
		2 altmezzo (Q682525)
	1 high voice (Q98116969)
		2 soprano (Q30903)
		2 child soprano (Q53829839)
		2 tenor (Q27914)
	1 operatic vocal (Q101436564)
	1 baritone (Q31687)
		2 Lyric baritone (Q8243255)
		2 acting baritone

In [33]:
tax.print_tree('Q1798603',depth)

0 string instrument (Q1798603)
	1 Guitarró (Q9001433)
	1 Kotamo (Q1785355)
	1 plucked string instrument (Q230262)
		2 hummel (Q2094724)
		2 veena (Q959769)
			3 Gayathri veena (Q56651492)
			3 vichitra veena (Q863302)
			3 bobbili veena (Q22137142)
			3 Sagar Veena (Q56063219)
			3 Mohan veena (Q6893623)
		2 bandora (Q16950185)
		2 đàn tranh (Q342234)
		2 Ang Kuy (Q530140)
		2 Adjalin (Q2824470)
		2 who are you? (Q717407)
		2 African harp (Q1343310)
			3 ennanga (Q5375216)
		2 espina (Q77073277)
		2 Janzi (Q52987174)
		2 chitarra battente (Q2297117)
		2 Q14295311 (Q14295311)
		2 virginal (Q922337)
		2 ronroco (Q1133096)
		2 lyre (Q201129)
			3 Q18451574 (Q18451574)
		2 tres (Q139727)
		2 Sintir (Q47165627)
		2 Home Swinger (Q3139624)
		2 Q21012217 (Q21012217)
		2 bandura (Q262414)
		2 chitarrone (Q2216381)
		2 Cura (Q1144423)
		2 Sagar Veena (Q56063219)
		2 Šargija (Q2451473)
		2 chillador (Q5099012)
		2 đàn bầu (Q342244)
		2 acoustic-electric guitar (Q988736)
		2 Marxophone (Q3296114)

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

In [34]:
tax.LCS('Q6012297','Q54634726')

{'Q186506'}

In [35]:
tax.LCS('Q5885030','Q6012300')

{'Q17172850'}

In [36]:
tax.LCS('Q27914','Q27914')

{'Q27914'}

In [37]:
tax.LCS('Q15336282','Q64166304')

{'Q105738', 'Q1798603', 'Q55724840'}

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [39]:
tax.dfs('Q186506','Q54634726')

['mezzo-soprano (Q186506)', 'light mezzo-soprano (Q54634726)']

In [40]:
tax.dfs('Q17172850','Q6012300')

['voice (Q17172850)', 'alto (Q6983813)', 'Q6012300 (Q6012300)']

In [41]:
tax.dfs('Q27914','Q27914')

['tenor (Q27914)']

In [43]:
tax.dfs('Q34379', 'Q105738')

['musical instrument (Q34379)', 'electrophone (Q105738)']

In [42]:
tax.dfs('Q34379','Q55724840')

['musical instrument (Q34379)',
 'chordophone (Q1051772)',
 'composite chordophones (Q19588495)',
 'lutes (Q1808578)',
 'handle lutes (Q30038759)',
 'necked lutes (Q55724833)',
 'necked box lutes (Q55724840)']

### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [44]:
tax.similarity('Q6012297','Q54634726')

(0.5,
 ['musical instrument (Q34379)',
  'voice (Q17172850)',
  'mezzo-soprano (Q186506)'],
 ['mezzo-soprano (Q186506)', 'dramatic mezzo-soprano (Q6012297)'],
 ['mezzo-soprano (Q186506)', 'light mezzo-soprano (Q54634726)'])

In [45]:
tax.similarity('Q186506','Q54634726')

(0.6666666666666666,
 ['musical instrument (Q34379)',
  'voice (Q17172850)',
  'mezzo-soprano (Q186506)'],
 ['mezzo-soprano (Q186506)'],
 ['mezzo-soprano (Q186506)', 'light mezzo-soprano (Q54634726)'])

In [46]:
tax.similarity('Q27914','Q27914')

(1.0,
 ['musical instrument (Q34379)',
  'voice (Q17172850)',
  'high voice (Q98116969)',
  'tenor (Q27914)'],
 ['tenor (Q27914)'],
 ['tenor (Q27914)'])

In [47]:
tax.similarity('Q76239', 'Q78987')

(0.42857142857142855,
 ['musical instrument (Q34379)',
  'chordophone (Q1051772)',
  'string instrument (Q1798603)',
  'plucked string instrument (Q230262)'],
 ['plucked string instrument (Q230262)', 'zither (Q76239)'],
 ['plucked string instrument (Q230262)',
  'plucked necked box lutes (Q57306162)',
  'guitar (Q6607)',
  'electric guitar (Q78987)'])

### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [200]:
tax.similarity('Q5994','Q5994') 
# Piano - piano: 1.0
# Son el mismo instrumento, por lo tanto tiene todo el sentido la similitud 1

(1.0,
 ['instrumento musical (Q34379)', 'teclado (Q52954)', 'piano (Q5994)'],
 ['piano (Q5994)'],
 ['piano (Q5994)'])

In [202]:
tax.similarity('Q5994', 'Q6607') 
# Piano - guitarra: 0.27
# Valor con bastante sentido, pues ambos son de cuerda e incluso de cuerda pulsada
# (razón por la cual si similitud no es ni 0 ni excesivamente baja).

(0.2727272727272727,
 ['instrumento musical (Q34379)',
  'cordófono (Q1051772)',
  'instrumento de cuerda (Q1798603)',
  'instrumento de cuerda pulsada (Q230262)'],
 ['instrumento de cuerda pulsada (Q230262)',
  'cítara (Q76239)',
  'board zithers (Q50829016)',
  'true board zithers (Q55724736)',
  'true board zithers with resonator (Q55724742)',
  'true board zithers with resonator box (Q4951628)',
  'piano (Q5994)'],
 ['instrumento de cuerda pulsada (Q230262)',
  'plucked necked box lutes (Q57306162)',
  'guitarra (Q6607)'])

In [203]:
tax.similarity('Q5994', 'Q78987') 
# Piano - guitarra eléctrica: 0.25
# es bastante razonable, pues es un poco menor que
# el piano y la guitarra normal, debido a que el piano no es electrico

(0.25,
 ['instrumento musical (Q34379)',
  'cordófono (Q1051772)',
  'instrumento de cuerda (Q1798603)',
  'instrumento de cuerda pulsada (Q230262)'],
 ['instrumento de cuerda pulsada (Q230262)',
  'cítara (Q76239)',
  'board zithers (Q50829016)',
  'true board zithers (Q55724736)',
  'true board zithers with resonator (Q55724742)',
  'true board zithers with resonator box (Q4951628)',
  'piano (Q5994)'],
 ['instrumento de cuerda pulsada (Q230262)',
  'plucked necked box lutes (Q57306162)',
  'guitarra (Q6607)',
  'guitarra eléctrica (Q78987)'])

In [205]:
tax.similarity('Q5994', 'Q11405')
# Piano - flauta: 0
# No se parecen en nada (el lcs es la raiz)

(0.0,
 ['instrumento musical (Q34379)'],
 ['instrumento musical (Q34379)', 'teclado (Q52954)', 'piano (Q5994)'],
 ['instrumento musical (Q34379)',
  'aerófono (Q659216)',
  'instrumento de viento (Q173453)',
  'flauta (Q11405)'])

In [206]:
tax.similarity('Q5994', 'Q11405')
# Piano - trompeta: 0 
# No se parecen en nada (el lcs es la raiz)

(0.0,
 ['instrumento musical (Q34379)'],
 ['instrumento musical (Q34379)', 'teclado (Q52954)', 'piano (Q5994)'],
 ['instrumento musical (Q34379)',
  'aerófono (Q659216)',
  'instrumento de viento (Q173453)',
  'flauta (Q11405)'])

In [207]:
tax.similarity('Q6607', 'Q78987') 
# Guitarra - guitarra electrica: 0.833
# Ambos instrumentos de cuerda y guitarras. Esperable

(0.8333333333333334,
 ['instrumento musical (Q34379)',
  'cordófono (Q1051772)',
  'instrumento de cuerda (Q1798603)',
  'instrumento de cuerda pulsada (Q230262)',
  'plucked necked box lutes (Q57306162)',
  'guitarra (Q6607)'],
 ['guitarra (Q6607)'],
 ['guitarra (Q6607)', 'guitarra eléctrica (Q78987)'])

In [208]:
tax.similarity('Q6607', 'Q11405')
# Guitarra - flauta: 0
# No son del mismo tipo ni se parecen, asi que es consistente la similitud 0

(0.0,
 ['instrumento musical (Q34379)'],
 ['instrumento musical (Q34379)',
  'cordófono (Q1051772)',
  'instrumento de cuerda (Q1798603)',
  'instrumento de cuerda pulsada (Q230262)',
  'plucked necked box lutes (Q57306162)',
  'guitarra (Q6607)'],
 ['instrumento musical (Q34379)',
  'aerófono (Q659216)',
  'instrumento de viento (Q173453)',
  'flauta (Q11405)'])

In [209]:
tax.similarity('Q6607', 'Q8338')
# Guitarra - trompeta: 0
# No son del mismo tipo ni se parecen, asi que es consistente la similitud

(0.0,
 ['instrumento musical (Q34379)'],
 ['instrumento musical (Q34379)',
  'cordófono (Q1051772)',
  'instrumento de cuerda (Q1798603)',
  'instrumento de cuerda pulsada (Q230262)',
  'plucked necked box lutes (Q57306162)',
  'guitarra (Q6607)'],
 ['instrumento musical (Q34379)',
  'aerófono (Q659216)',
  'instrumento de viento (Q173453)',
  'instrumento de viento-metal (Q180744)',
  'trompeta (Q8338)'])

In [210]:
tax.similarity('Q78987', 'Q11405' )
# Guitarra electrica - flauta: 0
# No se parecen en nada (el lcs es la raiz)

(0.0,
 ['instrumento musical (Q34379)'],
 ['instrumento musical (Q34379)',
  'electrófono (Q105738)',
  'electro-acoustic instruments (Q50821977)',
  'electro-acoustic chordophones (Q55738960)',
  'guitarra eléctrica (Q78987)'],
 ['instrumento musical (Q34379)',
  'aerófono (Q659216)',
  'instrumento de viento (Q173453)',
  'flauta (Q11405)'])

In [211]:
tax.similarity('Q78987', 'Q8338' )
# Guitarra electrica - trompeta: 0
# No son del mismo tipo ni se parecen, asi que es consistente la similitud 0

(0.0,
 ['instrumento musical (Q34379)'],
 ['instrumento musical (Q34379)',
  'electrófono (Q105738)',
  'electro-acoustic instruments (Q50821977)',
  'electro-acoustic chordophones (Q55738960)',
  'guitarra eléctrica (Q78987)'],
 ['instrumento musical (Q34379)',
  'aerófono (Q659216)',
  'instrumento de viento (Q173453)',
  'instrumento de viento-metal (Q180744)',
  'trompeta (Q8338)'])

In [212]:
tax.similarity('Q11405', 'Q8338' )
# Flauta - trompeta: 0.4 
# Ambos son de viento, y por ello la similitud es positiva.
# Sin embargo, son distintos instrumentos, y por ello tienen menor similitud que la guitarra y la guitarra electrica
# Sorprende algo quizá que su similitud sea mayor que la que hay entre el piano y la guitarra, siendo que el LCS en ese caso es, en teoría,
# menos que general que en este caso: instrumento de cuerda pulsada (nivel 4) y instrumento de viento (nivel 3).

(0.4,
 ['instrumento musical (Q34379)',
  'aerófono (Q659216)',
  'instrumento de viento (Q173453)'],
 ['instrumento de viento (Q173453)', 'flauta (Q11405)'],
 ['instrumento de viento (Q173453)',
  'instrumento de viento-metal (Q180744)',
  'trompeta (Q8338)'])